In [ ]:

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Padding
import pandas as pd
import soundfile as sf
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio #Import Audio method from IPython's Display Class
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Padding
import pandas as pd
import soundfile as sf
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Audio #Import Audio method from IPython's Display Class
from gtts import gTTS #Import Google Text to Speech
import os
import random


dataset1 = pd.read_csv("https://storage.googleapis.com/datathon2022/labels_dataset1_v2.csv")
#dataset1 = pd.read_csv("labels_dataset1_v2.csv")
# dataset1 = dataset1[dataset1['label']!= 'click']

df_whistle = dataset1.query("label == 'whistle'")
df_whistle.reset_index(inplace=True,drop=True)

df_click = dataset1.query("label == 'click'")
df_click.reset_index(inplace=True,drop=True)
df_click

df_rest= dataset1.query("label != 'click' and label != 'whistle' and label != 'volcano'")
df_rest.reset_index(inplace=True,drop=True)
df_rest

df = pd.concat([df_whistle[0:1500],df_click[0:1500], df_rest], ignore_index=True)
print(df.label.value_counts())

In [ ]:
df

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
df.to_csv('C:\\Users\\Oscar\\Desktop\\datos\\aux.csv',index = False)

In [ ]:
ruta_dst = 'C:\\Users\\Oscar\\Desktop\\datos\\augmented2'
for l in df['label'].unique():
  os.makedirs(f'{ruta_dst}\\{l}')

ruta_dst = 'C:\\Users\\Oscar\\Desktop\\datos\\original2'
for l in df['label'].unique():
  os.makedirs(f'{ruta_dst}\\{l}')

In [ ]:
#DATASET2
path_aug = "C:\\Users\\Oscar\\Desktop\\datos\\augmented2"
path_org = "C:\\Users\\Oscar\\Desktop\\datos\\original2"
audio_path = "C:\\Users\\Oscar\\Desktop\\dataset1\\"

augment1 = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
    Shift(min_fraction=-0.5, max_fraction=0.3, p=0.5),
    Padding(),
])

def process_audio_fragment(n_copies,n_times,label,filename,index, start, end, samplerate = 50000):
    """Play a fragment of an audio file.
    Args:
        filename: path to the audio file
        start: start of the fragment in samples
        end: end of the fragment in samples
        samplerate: samplerate to use when reading the file"""
    
    data, read_sr = sf.read(audio_path + filename + '.wav', start=start, stop=end)
    assert samplerate == read_sr, f"samplerate does not match {samplerate} (from file) != {read_sr} (function parameter)"

    fig = plt.figure(figsize=[1,1])
    
    # This is to get rid of the axes and only get the picture 
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    
    # This is the melspectrogram from the decibels with a linear relationship
    S = librosa.feature.melspectrogram(y=data, sr=samplerate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), y_axis='linear')
    
    # Here we choose the path and the name to save the file, we will change the path when
    # using the function for train, val and test to make the function easy to use and output
    # the images in different folders to use later with a generator
    
    for i in range(n_copies):
        if random.random() < 0.5 :
            file_spectrogram  = path_aug + '\\' + label + '\\duplicated'+str(i)+ str(index) + '.jpg'
            plt.savefig(file_spectrogram, dpi=500, bbox_inches='tight',pad_inches=0)
        else:
            file_spectrogram  = path_org + '\\' + label + '\\duplicated'+str(i)+ str(index) + '.jpg'
            plt.savefig(file_spectrogram, dpi=500, bbox_inches='tight',pad_inches=0)
        
    
    # Here we close the image because otherwise we get a warning saying that the image stays
    # open and consumes memory
    plt.close()
    for i in range(n_times):
        augmented_samples = augment1(samples=np.float32(data), sample_rate=read_sr)
        assert samplerate == read_sr, f"samplerate does not match {samplerate} (from file) != {read_sr} (function parameter)"

        fig = plt.figure(figsize=[1,1])
        
        # This is to get rid of the axes and only get the picture 
        ax = fig.add_subplot(111)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_frame_on(False)
        
        # This is the melspectrogram from the decibels with a linear relationship
        S = librosa.feature.melspectrogram(y=augmented_samples, sr=samplerate)
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max), y_axis='linear')
        
        # Here we choose the path and the name to save the file, we will change the path when
        # using the function for train, val and test to make the function easy to use and output
        # the images in different folders to use later with a generator
        file_spectrogram  = path_aug + '\\' + label + '\\' + 'augmented'+str(i)+ '_'+ str(index) + '.jpg'
        # Here we finally save the image file choosing the resolution 
        plt.savefig(file_spectrogram, dpi=500, bbox_inches='tight',pad_inches=0)
        
        # Here we close the image because otherwise we get a warning saying that the image stays
        # open and consumes memory
        plt.close()

def process_annotation_from_df(row,index,margin: int = 0, samplerate = 50000):
    """Play a fragment of a wav file in a jupyter notebook.
    Args:
        row: a row of a pandas dataframe with the following columns:
            - path: path to the wav file
            - offset: offset in seconds
            - duration: duration in seconds
        margin: margin in seconds to add to the start and end of the fragment
        samplerate: samplerate to use when reading the file
        """
    m = margin * samplerate # margin in samples
    start = max(int(np.floor(row['start'] - m)), 0)
    end = int(np.ceil(row['start'] + row['duration'] * samplerate + m))
    filename = row['path']
    if index % 50 == 0:
      print(index)
    if row['label'] == 'whistle' or row['label'] == 'click':
        n_times= 0
        n_copies = 1
    elif row['label'] == 'allfreq':
        n_times = 15
        n_copies = 25
    else:
        n_times = 2
        n_copies = 2
    
    process_audio_fragment(n_copies,n_times,row['label'],filename,index, start, end, samplerate)

In [ ]:
for index, row in df.iterrows():
    process_annotation_from_df(row,index)
   